In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[9],7
commitnumber,1013f91,5014a19
conc,-,3.2e-07
dv,-,0.001
klin,-,2.22e-20
molecule,{'n2o': 3.2e-07},n2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.016431                    1  0.001639
0.000750        2 -0.014944                    2  0.002353
0.001052        3 -0.011700                    3  0.004365
0.001476        4 -0.008104                    4  0.006414
0.002070        5 -0.004174                    5  0.008676
0.002904        6  0.000116                    6  0.011153
0.004074        7  0.004761                    7  0.013850
0.005714        8  0.009797                    8  0.016817
0.008015        9  0.015244                    9  0.020056
0.011243       10  0.021118                   10  0.023592
0.015771       11  0.027436                   11  0.027444
0.022122       12  0.034215                   12  0.031636
0.031031       13  0.041428                   13  0.036177
0.043528       14  0.048107                   14  0.040503
0.061057       15  0.053251                   15  0.044060
0.085645       16  0.057585                   16  0.047297
0.120136       17  0.061846                   17  0.050676
0.168516       18  0.066039                   18  0.054219
0.236378       19  0.070183                   19  0.057912
0.331549       20  0.074368                   20  0.061776
0.465100       21  0.078206                   21  0.065402
0.652400       22  0.071907                   22  0.062473
0.915100       23  0.053797                   23  0.051484
1.283650       24  0.036096                   24  0.039695
1.800600       25  0.022900                   25  0.029592
2.525700       26  0.013582                   26  0.020980
3.542800       27  0.007307                   27  0.013741
4.969550       28  0.004014                   28  0.008421
6.970850       29  0.003025                   29  0.005072
9.778100       30  0.002615                   30  0.002727
13.715850      31  0.001991                   31  0.000834
19.239350      32  0.001131                   32 -0.000619
26.987250      33  0.000769                   33 -0.001215
37.855300      34  0.001009                   34 -0.001139
53.100050      35  0.001106                   35 -0.001187
73.887500      36  0.000992                   36 -0.001578
97.662500      37  0.000664                   37 -0.002281
121.437500     38 -0.000105                   38 -0.003328
145.212500     39 -0.001153                   39 -0.004554
168.987500     40 -0.002217                   40 -0.005738
192.762500     41 -0.003343                   41 -0.006888
216.537500     42 -0.004571                   42 -0.008019
240.312500     43 -0.005943                   43 -0.009132
264.087500     44 -0.007516                   44 -0.010233
287.862500     45 -0.009212                   45 -0.011222
311.637500     46 -0.009979                   46 -0.011427
335.412500     47 -0.009184                   47 -0.010481
359.187500     48 -0.007719                   48 -0.008937
382.962500     49 -0.006203                   49 -0.007231
406.737500     50 -0.004633                   50 -0.005410
430.512500     51 -0.002986                   51 -0.003487
454.287500     52 -0.001248                   52 -0.001478
478.062500     53  0.000582                   53  0.000627
501.837500     54  0.002495                   54  0.002790
525.612500     55  0.004478                   55  0.004994
549.387500     56  0.006503                   56  0.007216
573.162500     57  0.008539                   57  0.009428
596.937500     58  0.010548                   58  0.011594
620.712500     59  0.012493                   59  0.013671
644.487500     60  0.014342                   60  0.015658
668.262500     61  0.016076                   61  0.017505
692.037500     62  0.017696                   62  0.019222
715.812500     63  0.019214                   63  0.020817
739.587500     64  0.020653                   64  0.022304
763.362500     65  0.022041                   65  0.023697
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -8.678711  0.000000 -8.678711 -8.629749  9.398477e-08   
0.000624    2     -8.678713  0.000001 -8.678712 -8.629750  6.751157e-07   
0.000876    3     -8.678714  0.000001 -8.678713 -8.629750  9.270130e-07   
0.001229    4     -8.678715  0.000002 -8.678713 -8.629750  1.323174e-06   
0.001723    5     -8.678717  0.000003 -8.678714 -8.629750  1.936309e-06   
0.002417    6     -8.678719  0.000005 -8.678714 -8.629751  2.886452e-06   
0.003391    7     -8.678721  0.000007 -8.678714 -8.629751  4.359269e-06   
0.004757    8     -8.678725  0.000011 -8.678713 -8.629751  6.640988e-06   
0.006672    9     -8.678728  0.000017 -8.678711 -8.629751  1.017631e-05   
0.009359    10    -8.678733  0.000027 -8.678706 -8.629750  1.564969e-05   
0.013128    11    -8.678738  0.000041 -8.678697 -8.629748  2.411629e-05   
0.018415    12    -8.678743  0.000063 -8.678680 -8.629743  3.719953e-05   
0.025830    13    -8.678746  0.000097 -8.678650 -8.629736  5.739362e-05   
0.036232    14    -8.678747  0.000148 -8.678599 -8.629722  8.851445e-05   
0.050823    15    -8.678741  0.000226 -8.678515 -8.629700  1.358891e-04   
0.071291    16    -8.678727  0.000340 -8.678386 -8.629664  2.066020e-04   
0.100000    17    -8.678696  0.000506 -8.678190 -8.629607  3.111769e-04   
0.140271    18    -8.678641  0.000745 -8.677896 -8.629520  4.656608e-04   
0.196760    19    -8.678542  0.001088 -8.677454 -8.629385  6.936253e-04   
0.275997    20    -8.678370  0.001575 -8.676795 -8.629177  1.029420e-03   
0.387100    21    -8.678078  0.002261 -8.675816 -8.628858  1.522853e-03   
0.543100    22    -8.677583  0.003212 -8.674371 -8.628369  2.243133e-03   
0.761700    23    -8.676884  0.004375 -8.672509 -8.627710  3.202150e-03   
1.068500    24    -8.676110  0.005556 -8.670554 -8.626963  4.326370e-03   
1.498800    25    -8.675335  0.006621 -8.668714 -8.626182  5.568574e-03   
2.102400    26    -8.674589  0.007512 -8.667077 -8.625398  6.900801e-03   
2.949000    27    -8.673917  0.008202 -8.665714 -8.624664  8.270573e-03   
4.136600    28    -8.673401  0.008714 -8.664687 -8.624061  9.601166e-03   
5.802500    29    -8.673101  0.009207 -8.663894 -8.623655  1.085739e-02   
8.139200    30    -8.673010  0.009953 -8.663057 -8.623484  1.208978e-02   
11.417000   31    -8.673202  0.011160 -8.662042 -8.623686  1.335137e-02   
16.014700   32    -8.673938  0.012981 -8.660958 -8.624614  1.473345e-02   
22.464000   33    -8.675747  0.015653 -8.660094 -8.626864  1.651016e-02   
31.510500   34    -8.679149  0.019880 -8.659270 -8.631129  1.947351e-02   
44.200100   35    -8.684572  0.026820 -8.657752 -8.638281  2.491321e-02   
62.000000   36    -8.693043  0.037623 -8.655420 -8.650102  3.423131e-02   
85.775000   37    -8.705720  0.053095 -8.652625 -8.668742  4.842647e-02   
109.550000  38    -8.719948  0.069193 -8.650755 -8.690427  6.368706e-02   
133.325000  39    -8.736235  0.085184 -8.651051 -8.715382  7.926814e-02   
157.100000  40    -8.755093  0.100793 -8.654300 -8.743765  9.482558e-02   
180.875000  41    -8.776703  0.116158 -8.660545 -8.775473  1.103714e-01   
204.650000  42    -8.801293  0.131332 -8.669961 -8.810432  1.259287e-01   
228.425000  43    -8.829171  0.146338 -8.682833 -8.848609  1.415185e-01   
252.200000  44    -8.860752  0.161183 -8.699569 -8.889956  1.571446e-01   
275.975000  45    -8.896617  0.175880 -8.720737 -8.934441  1.728078e-01   
299.750000  46    -8.937347  0.190666 -8.746680 -8.981894  1.886525e-01   
323.525000  47    -8.981974  0.207188 -8.774786 -9.031170  2.057431e-01   
347.300000  48    -9.028240  0.227589 -8.800651 -9.080562  2.256144e-01   
371.075000  49    -9.074829  0.252438 -8.822391 -9.129080  2.489613e-01   
394.850000  50    -9.121275  0.281415 -8.839859 -9.176335  2.758476e-01   
418.625000  51    -9.167198  0.314290 -8.852908 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')